In [1]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np

In [2]:
# root@ns-Precision-Tower-7910:/home/sawyer_ws/src# rostopic echo  /io/end_effector/right_gripper/command 
# time: 
#   secs: 1694119441
#   nsecs: 878112554
# op: "set"
# args: "{\"signals\": {\"position_m\": {\"data\": [0.041667], \"format\": {\"type\": \"float\"\
#   }}}}"
# ---
# time: 
#   secs: 1694119446
#   nsecs: 234554767
# op: "set"
# args: "{\"signals\": {\"position_m\": {\"data\": [0.0], \"format\": {\"type\": \"float\"\
#   }}}}"
# ---


In [2]:
from intera_core_msgs.msg import IOComponentCommand
from std_msgs.msg import Bool

In [7]:
pub = node.create_publisher(IOComponentCommand, '/io/end_effector/right_gripper/command', 3)

calibrate2 = '{"signals": {"calibrate": {"data": [true], "format": {"type": "bool"}}}}'
calibrate1 = '{"signals": {"dead_zone_m": {"data": [0.001], "format": {"type": "float"}}}}'
close = '{"signals": {"position_m": {"data": [0.0], "format": {"type": "float"}}}}'
open = '{"signals": {"position_m": {"data": [0.041667], "format": {"type": "float"}}}}'


In [8]:
def calibrate_gripper():
    command = IOComponentCommand()
    # command.time = rospy.rostime.get_rostime()
    command.op = "set"
    command.args = calibrate1
    pub.publish(command)
    command.args = calibrate2
    pub.publish(command)

def callback_gripper(msg):
    command = IOComponentCommand()
    # command.time = rospy.rostime.get_rostime()
    if msg.data:
        command.op = "set"
        command.args = open
    else:
        command.op = "set"
        command.args = close
    pub.publish(command)

In [15]:
command = IOComponentCommand()
# command.time = rospy.rostime.get_rostime()
command.header.stamp = rclpy.clock.Clock().now().to_msg()
command.op = "set"
command.args = open

pub.publish(command)

In [17]:
command = IOComponentCommand()
# command.time = rospy.rostime.get_rostime()
command.header.stamp = rclpy.clock.Clock().now().to_msg()
command.op = "set"
command.args = close

pub.publish(command)

In [6]:
pub_gripper = node.create_publisher(Bool, '/gripper_command', 2)

In [9]:
msgB = Bool()
msgB.data = True
pub_gripper.publish(msgB)

In [8]:
msgB = Bool()
msgB.data = False
pub_gripper.publish(msgB)

In [ ]:
# ros2 topic pub /gripper_command std_msgs/msg/Bool "data: true"

In [3]:
class ROSInterface:
    def __init__(self, node, robot):
        self.joy_msg = None
        self.joint_states_msg = None
        self.robot = robot
        self.node = node
        self.jointMap = {name: ind for ind, name in enumerate(self.robot.jointNames)} 

    def joy_callback(self, msg):
        self.joy_msg = msg

    def joint_states_callback(self, msg):
        self.joint_states_msg = msg
        q = self.robot.getJoints()
        print('joint_states:', q)
        for ind, name in enumerate(msg.name):
            if name in self.jointMap:
                q[self.jointMap[name]] = msg.position[ind]
        self.robot.setJoints(q)

    def spin_thread(self):
        self.node.create_subscription(JointState, 'robot/joint_states', self.joint_states_callback, 10)
        self.node.create_subscription(Joy, '/joy', self.joy_callback, 10)
        self.node.create_subscription(Bool, '/gripper_command',callback_gripper, 10)
        rclpy.spin(self.node)

In [4]:
trigger = lambda msg: msg.buttons[0]
mode = lambda msg: msg.buttons[6] == 0
forward = lambda msg: (-msg.axes[1] / 5) * mode(msg)
right = lambda msg: (-(msg.axes[0] - 0.12488976866006851 * 0) / 5) * mode(msg)
up = lambda msg: (msg.axes[2] / 5) * mode(msg)
roll = lambda msg: (msg.axes[0] / 1) * (1 - mode(msg))
pitch = lambda msg: (-msg.axes[1] / 1) * (1 - mode(msg))
yaw = lambda msg: (msg.axes[2] / 1) * (1 - mode(msg))

In [5]:
rclpy.init()
robot = URDFModel('/home/ns/sawyer_robot_ros2/src/teleop_script/sawyer.urdf')
node = Node('pub_dbg')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

In [ ]:


ros_interface.spin_thread()
# t1 = Thread(target=ros_interface.spin_thread)
# t1.start()

In [8]:
msg=ros_interface.joint_states_msg
msg 

sensor_msgs.msg.JointState(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1694111890, nanosec=504633821), frame_id=''), name=['head_pan', 'right_j0', 'right_j1', 'right_j2', 'right_j3', 'right_j4', 'right_j5', 'right_j6', 'torso_t0'], position=[-1.0600302734375, 0.595087890625, 0.215681640625, -2.4052880859375, 1.817673828125, -0.8160146484375, -1.1161943359375, 3.0333583984375, 0.0], velocity=[-0.001, -0.001, -0.001, -0.001, -0.001, -0.001, -0.001, -0.001, 0.0], effort=[0.04, -1.22, -26.92, -3.712, -0.168, -0.8, 1.6, 0.044, 0.0])

In [12]:
msg.header

std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1694111890, nanosec=504633821), frame_id='')

In [13]:
msg.name

['head_pan',
 'right_j0',
 'right_j1',
 'right_j2',
 'right_j3',
 'right_j4',
 'right_j5',
 'right_j6',
 'torso_t0']

In [14]:
msg.position

array('d', [-1.0600302734375, 0.595087890625, 0.215681640625, -2.4052880859375, 1.817673828125, -0.8160146484375, -1.1161943359375, 3.0333583984375, 0.0])

In [15]:
msg.velocity

array('d', [-0.001, -0.001, -0.001, -0.001, -0.001, -0.001, -0.001, -0.001, 0.0])

In [16]:
msg.effort

array('d', [0.04, -1.22, -26.92, -3.712, -0.168, -0.8, 1.6, 0.044, 0.0])

In [18]:
ros_interface.robot.getJoints()

array([ 0.59508789, -1.06003027,  0.21568164, -2.40528809,  1.81767383,
       -0.81601465, -1.11619434,  3.0333584 ,  0.        ,  0.        ])

In [19]:
ros_interface.robot.jointNames

['right_j0',
 'head_pan',
 'right_j1',
 'right_j2',
 'right_j3',
 'right_j4',
 'right_j5',
 'right_j6',
 'right_gripper_l_finger_joint',
 'right_gripper_r_finger_joint']

In [ ]:


I = np.eye(q.shape[0])
alpha = .0001
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)
cmd_msg = JointCommand()
cmd_msg.names = robot.jointNames
cmd_msg.mode = cmd_msg.VELOCITY_MODE
gripper_msg = JointCommand()
gripper_msg.mode = cmd_msg.POSITION_MODE
gripper_msg.names = ['right_gripper_l_finger_joint', 'right_gripper_r_finger_joint']
gripper_msg.position = [0.0, 0.0]
last_trigger = 0
while rclpy.ok():
    if ros_interface.joy_msg and ros_interface.joint_states_msg:
        print(ros_interface.joy_msg)
        m = ros_interface.joy_msg
        xd = np.array([forward(m), right(m), up(m), roll(m), pitch(m), yaw(m)])
        J = robot.getJacobian('camera_link')
        qd = np.linalg.inv(J.T @ J + alpha * I) @ J.T @ xd
        cmd_msg.velocity = list(qd)

        cmd_pub.publish(cmd_msg)
        if trigger(ros_interface.joy_msg) == 1 and last_trigger == 0:
            if gripper_msg.position[0] == 0.0:
                gripper_msg.position = [.03, -.03]
            else:
                gripper_msg.position = [0.0, 0.0]
            cmd_pub.publish(gripper_msg)
            last_trigger = 1
        elif trigger(ros_interface.joy_msg) == 0:
            last_trigger = 0
        time.sleep(.1)